### Predicted a list of ZTF objects.

In [1]:
from generate_annotator import *
import pandas as pd 
import lasair, os
import settings
import numpy as np 


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



%6|1718743951.282|FAIL|rdkafka#consumer-1| [thrd:kafka.lsst.ac.uk:9092/bootstrap]: kafka.lsst.ac.uk:9092/bootstrap: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY)
%6|1718743951.506|FAIL|rdkafka#consumer-1| [thrd:kafka.lsst.ac.uk:9092/bootstrap]: kafka.lsst.ac.uk:9092/bootstrap: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY, 1 identical error(s) suppressed)


Annotated 0 of 0 objects


In [8]:

def collect_data_from_lasair(objectId, objectInfo, band = 'r'):
    if band == 'g':
        fid = 1
    elif band == 'r':
        fid = 2

    candidates = objectInfo['candidates']
    
    candidates = [x for x in candidates if 'image_urls' in x.keys() and x['fid'] == fid]
    # print(candidates)
    
    disdate = objectInfo['objectData']['discMjd']
    discFilter = objectInfo['objectData']['discFilter']

    if len(candidates) >= 1:
        mags = np.array([m['magpsf']for m in candidates])
        idx = np.argmin(mags)

        if discFilter == band:
            discMag = objectInfo['objectData']['discMag']
            discMag = float(discMag.strip(r'\u')[0])
        else:
            discMag = candidates[-1]['magpsf']

        flag = False
        
        while flag is False and idx <= len(candidates)-1 and idx >= 0:
      
            peak_urls = candidates[idx]['image_urls']
            science_url = peak_urls['Science']
            template_url = peak_urls['Template']
            # difference_url = peak_urls['Difference']

            obsjd_path = ztf_image_pipeline.create_path(NEEDLE_OBJ_PATH, objectId)
            sci_fname = "sci_ztf_peak.fits"
            sci_filename = obsjd_path + '/' + sci_fname
            ref_fname = 'ref_ztf_peak.fits'
            ref_filename = obsjd_path + '/' + ref_fname

            # if not os.path.exists(sci_filename):
            os.system("curl -o %s %s" % (sci_filename, science_url))
            os.system("curl -o %s %s" % (ref_filename, template_url))
            
            if os.path.getsize(sci_filename) < 800 or os.path.getsize(ref_filename) < 800: 
                idx += 1
            else:
                flag = True
                break
                
        if flag:

            img_data = get_obj_image(sci_filename, ref_filename, None, BClassifier)
            if img_data is not None:    
                host_ra, host_dec = objectInfo['sherlock']['raDeg'], objectInfo['sherlock']['decDeg']
                PS1catalog_host(_id = objectId, _ra = host_ra, _dec = host_dec, save_path = NEEDLE_OBJ_PATH + '/hosts')
                host_meta = build_dataset.add_host_meta(objectId, host_path = NEEDLE_OBJ_PATH + '/hosts', only_complete = True)
                sherlock_meta = [objectInfo['sherlock']['separationArcsec']]
                if host_meta is None:
                    print('object %s host meta not found, use NEEDLE-T\n' % objectId)
                    meta_data = get_obj_meta(candidates, idx, disdate, discMag, None)
                    meta_data = scaling_meta(meta_data, NEEDLE_PATH_T)
                    return img_data, meta_data, 'T'
             
                else:
                    meta_data = get_obj_meta(candidates, idx, disdate, discMag, host_meta[1]) + host_meta + sherlock_meta
                    meta_data = scaling_meta(meta_data, NEEDLE_PATH_TH)
                    return img_data, meta_data, 'TH'
            else:

                return None, None, None
        else:
     
            return None, None, None

    else:
       
        return None, None, None
    
def predict_targeted_ZTF(objectId, L,  threshold = 0.75):

    objectInfo = L.objects([objectId])[0]
    classification = None
    probs = None
    # print(objectInfo)
    if objectInfo is None:
        print('objectInfo is None.')
        return classification, probs
    else:
        img_data, meta_data, findhost = collect_data_from_lasair(objectId, objectInfo, band = 'r')

        if img_data is None or meta_data is None:
            print('image data is None or meta data is None.')
            return classification, probs
        else:
            results = needle_prediction(img_data, meta_data, findhost)
            probs = 'SN: %f, SLSN-I: %f, TDE: %f' % (results[0][0], results[0][1], results[0][2])
            
            if np.max(results[0]) >= threshold:
                classification = LABEL_LIST[np.argmax(results[0])]
            else:
                classification = 'unclear'
            return classification, probs
    

In [9]:
objectId = 'ZTF24aakzjyd'
L = lasair.lasair_client(settings.API_TOKEN)
predict_targeted_ZTF(objectId, L,  threshold = 0.75)

1/1 [==============================] - 0s 13ms/step


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20160  100 20160    0     0   268k      0 --:--:-- --:--:-- --:--:--  289k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20160  100 20160    0     0   312k      0 --:--:-- --:--:-- --:--:--  328k


1/1 [==============================] - 0s 12ms/step
https://catalogs.mast.stsci.edu/api/v0.1/panstarrs/dr2/stack.json?ra=0.0&dec=0.0&radius=0.00139&columns=[raStack,decStack,gPSFMag,gPSFMagErr,rPSFMag,rPSFMagErr,iPSFMag,iPSFMagErr,zPSFMag,zPSFMagErr,yPSFMag,yPSFMagErr, gApMag,gApMagErr,rApMag,rApMagErr,iApMag,iApMagErr,zApMag,zApMagErr,yApMag,yApMagErr,yKronMag]&nDetections.gte=6&pagesize=10000
object ZTF24aakzjyd host meta not found, use NEEDLE-T

1/1 [==============================] - 0s 136ms/step


('unclear', 'SN: 0.192751, SLSN-I: 0.279074, TDE: 0.528175')

%6|1718744329.666|FAIL|rdkafka#consumer-1| [thrd:kafka.lsst.ac.uk:9092/bootstrap]: kafka.lsst.ac.uk:9092/bootstrap: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY, 4 identical error(s) suppressed)
%6|1718744359.725|FAIL|rdkafka#consumer-1| [thrd:kafka.lsst.ac.uk:9092/bootstrap]: kafka.lsst.ac.uk:9092/bootstrap: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY, 3 identical error(s) suppressed)
%6|1718744389.788|FAIL|rdkafka#consumer-1| [thrd:kafka.lsst.ac.uk:9092/bootstrap]: kafka.lsst.ac.uk:9092/bootstrap: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener